In [4]:
import sys

sys.path.insert(0, '/home/caron/Bureau/Model/HexagonalScale/ST-GNN-for-wildifre-prediction/Prediction')

from firecastnet.seasfire.backbones.graphcast.graph_builder import *

icospheres_graph_path="/home/caron/Bureau/Model/HexagonalScale/firecastnet/icospheres/icospheres_0_1_2_3_4_5_6.json.gz"

In [5]:
import xarray as xr

# Get xarray dataset from source
def get_xarray_dataset(path_to_dataset='SeasFire.zarr'):
    ds = xr.open_zarr(Path.home() / path_to_dataset)
    return ds

In [3]:
import torch

torch.__version__

'2.4.0+cu121'

In [6]:
import geopandas as gpd
import pickle
from pathlib import Path

path = Path('/home/caron/Bureau/Model/HexagonalScale/ST-GNN-for-wildifre-prediction/Prediction/GNN/firemen/firepoint/2x2/train')

graph = pickle.load(open(path / 'graph_7_risk-size-watershed_graph.pkl', 'rb'))

#geo = gpd.read_file('regions/firepoint/bdiff/regions.geojson')

#ds = read_object('france.pkl', Path('/media/caron/X9 Pro/travaille/Thèse/csv/SeasFire/seasfire_v0.3/'))

#ds = get_xarray_dataset(path_to_dataset='/media/caron/X9 Pro/travaille/Thèse/csv/SeasFire/seasfire_v0.3/SeasFireCube_v3.zarr')

/home/caron/Bureau/.conda/lib/python3.9/site-packages/torch/cuda/__init__.py:128: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


Nom de l'hôte : caron-Precision-7780
Adresse IP locale : 127.0.1.1
Nom de l'hôte : caron-Precision-7780
Adresse IP locale : 127.0.1.1


In [7]:
from config import *

lats = graph.nodes[:, latitude_index]
lons = graph.nodes[:, longitude_index]
#lats = geo.latitude.values
#lons = geo.longitude.values
#lats = ds.latitude.values
#lons = ds.longitude.values

g_lat_lon_grid = np.asarray([lats, lons])
g_lat_lon_grid.shape
g_lat_lon_grid = torch.Tensor(g_lat_lon_grid)

g_max_lat = np.max(lats)
g_min_lat = np.min(lats)
g_max_lon = np.max(lons)
g_min_lon = np.min(lons)

g_sp_res = 0.25

Nom de l'hôte : caron-Precision-7780
Adresse IP locale : 127.0.1.1


In [8]:
#graph._plot(graph.nodes, dir_output=Path('./'))

In [9]:
# create the lat_lon_grid for the graph
"""_g_latitudes = torch.from_numpy(
    np.arange(
        g_max_lat, g_min_lat - (g_sp_res / 2), -g_sp_res, dtype=np.float32
    )
)
_g_lat_dim = _g_latitudes.size(0)
logger.info("Latitude dimension (graph) = {}".format(_g_lat_dim))

_g_longitudes = torch.from_numpy(
    np.arange(g_min_lon, g_max_lon + (g_sp_res / 2), g_sp_res, dtype=np.float32)
)
_g_lon_dim = _g_longitudes.size(0)
logger.info("Longitude dimension (graph) = {}".format(_g_lon_dim))

_g_lat_lon_grid = torch.stack(
    torch.meshgrid(_g_latitudes, _g_longitudes, indexing="ij"), dim=-1
)"""

#g_lat_lon_grid = _g_lat_lon_grid

"""g_lat_lon_grid = torch.stack(
    torch.meshgrid(torch.Tensor(lats), torch.Tensor(lons), indexing="ij"), dim=-1
)"""

'g_lat_lon_grid = torch.stack(\n    torch.meshgrid(torch.Tensor(lats), torch.Tensor(lons), indexing="ij"), dim=-1\n)'

In [10]:
"""import pandas as pd

_g_lat_lon_grid = g_lat_lon_grid.reshape(-1,2)

df = pd.DataFrame(index=np.arange(0, _g_lat_lon_grid.shape[0]))
df['latitude'] = _g_lat_lon_grid[:, 0]
df['longitude'] = _g_lat_lon_grid[:, 1]


gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.longitude, df.latitude))
gdf.plot()"""

"import pandas as pd\n\n_g_lat_lon_grid = g_lat_lon_grid.reshape(-1,2)\n\ndf = pd.DataFrame(index=np.arange(0, _g_lat_lon_grid.shape[0]))\ndf['latitude'] = _g_lat_lon_grid[:, 0]\ndf['longitude'] = _g_lat_lon_grid[:, 1]\n\n\ngdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.longitude, df.latitude))\ngdf.plot()"

In [11]:
graph = GraphBuilder(
            icospheres_graph_path=icospheres_graph_path,
            lat_lon_grid=g_lat_lon_grid,
        )

2025-02-19 17:23:42,772 [INFO ]  Opened pre-computed graph from /home/caron/Bureau/Model/HexagonalScale/firecastnet/icospheres/icospheres_0_1_2_3_4_5_6.json.gz.
2025-02-19 17:23:42,772 [INFO ]  Opened pre-computed graph from /home/caron/Bureau/Model/HexagonalScale/firecastnet/icospheres/icospheres_0_1_2_3_4_5_6.json.gz.
2025-02-19 17:23:42,773 [INFO ]  Will use max_order=6 icospheres
2025-02-19 17:23:42,773 [INFO ]  Will use max_order=6 icospheres


In [12]:
cartesian_grid = latlon2xyz(g_lat_lon_grid)
cartesian_grid

tensor([[0.478, 0.498, 0.723],
        [0.992, 0.098, 0.086]])

In [13]:
g_lat_lon_grid.permute(0,1).shape

torch.Size([2, 33])

In [14]:
graph.icospheres['order_0_vertices']

array([[-0.52573111,  0.85065081,  0.        ],
       [ 0.52573111,  0.85065081,  0.        ],
       [-0.52573111, -0.85065081,  0.        ],
       [ 0.52573111, -0.85065081,  0.        ],
       [ 0.        , -0.52573111,  0.85065081],
       [ 0.        ,  0.52573111,  0.85065081],
       [ 0.        , -0.52573111, -0.85065081],
       [ 0.        ,  0.52573111, -0.85065081],
       [ 0.85065081,  0.        , -0.52573111],
       [ 0.85065081,  0.        ,  0.52573111],
       [-0.85065081,  0.        , -0.52573111],
       [-0.85065081,  0.        ,  0.52573111]])

In [15]:
import dgl
import networkx as nx
import torch
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

def plot_dgl_graph_3d(graph, title="DGL Graph 3D"):
    """Affiche un graphe DGL en 3D avec une grande figure."""
    
    # Trouver la clé correcte des positions
    if "pos" in graph.ndata:
        pos = graph.ndata["pos"].numpy()
    elif "x" in graph.ndata:
        pos = graph.ndata["x"].numpy()
    else:
        raise KeyError("Aucune donnée de position trouvée dans le graphe !")

    nx_graph = graph.to_networkx()

    fig = plt.figure(figsize=(15, 12))  # Augmenter la taille de la figure
    ax = fig.add_subplot(111, projection="3d")

    for src, dst in nx_graph.edges():
        x_vals = [pos[src][0], pos[dst][0]]
        y_vals = [pos[src][1], pos[dst][1]]
        z_vals = [pos[src][2], pos[dst][2]]
        ax.plot(x_vals, y_vals, z_vals, "gray", alpha=0.6)

    ax.scatter(pos[:, 0], pos[:, 1], pos[:, 2], c="blue", s=50)  # Augmenter la taille des nœuds

    ax.set_title(title, fontsize=18)  # Augmenter la taille du titre
    plt.show()

def plot_dgl_hetero_graph_3d(graph, node_types, edge_type, title="DGL Hetero Graph 3D"):
    """Affiche un graphe DGL hétérogène en 3D avec une grande figure.
    
    Args:
        graph (dgl.DGLHeteroGraph): Le graphe hétérogène à afficher.
        node_types (tuple): Tuple des types de nœuds (ex: ("grid", "mesh")).
        edge_type (tuple): Tuple représentant le type d’arête (ex: ("grid", "g2m", "mesh")).
        title (str): Titre du graphe.
    """

    src_type, dst_type = node_types  # Ex: ("grid", "mesh")

    # Vérifier si le type d’arête est présent dans le graphe
    #if edge_type not in graph.metagraph().edges:
    #    raise ValueError(f"L'arête {edge_type} n'existe pas dans le graphe !")

    # Vérifier la présence des positions des nœuds
    if "pos" in graph.nodes[src_type].data and "pos" in graph.nodes[dst_type].data:
        src_pos = graph.nodes[src_type].data["pos"].numpy()
        dst_pos = graph.nodes[dst_type].data["pos"].numpy()
    else:
        raise KeyError("Les positions 'pos' ne sont pas définies pour certains nœuds.")

    # Convertir en graphe NetworkX pour visualisation
    nx_graph = dgl.to_networkx(graph)

    fig = plt.figure(figsize=(15, 12))
    ax = fig.add_subplot(111, projection="3d")

    # Dessiner les arêtes
    for src, dst in zip(*graph.edges):
        x_vals = [src_pos[src][0], dst_pos[dst][0]]
        y_vals = [src_pos[src][1], dst_pos[dst][1]]
        z_vals = [src_pos[src][2], dst_pos[dst][2]]
        ax.plot(x_vals, y_vals, z_vals, "gray", alpha=0.6)

    # Dessiner les nœuds sources et destinations avec des couleurs distinctes
    ax.scatter(src_pos[:, 0], src_pos[:, 1], src_pos[:, 2], c="blue", s=50, label=src_type)
    ax.scatter(dst_pos[:, 0], dst_pos[:, 1], dst_pos[:, 2], c="red", s=50, label=dst_type)

    ax.set_title(title, fontsize=18)
    ax.legend()
    plt.show()


In [16]:
graph_mesh = graph.create_mesh_graph()
graph2mesh, graph_mesh = graph.create_g2m_graph(graph_mesh)
graphmesh2graph = graph.create_m2g_graph()

#plot_dgl_graph_3d(graph_mesh)

2025-02-19 17:23:42,887 [INFO ]  Creating bi-directional mesh graph
2025-02-19 17:23:42,887 [INFO ]  Creating bi-directional mesh graph
2025-02-19 17:23:43,665 [INFO ]  mesh graph=Graph(num_nodes=40962, num_edges=327660,
      ndata_schemes={'x': Scheme(shape=(3,), dtype=torch.float32)}
      edata_schemes={'x': Scheme(shape=(4,), dtype=torch.float32)})
2025-02-19 17:23:43,665 [INFO ]  mesh graph=Graph(num_nodes=40962, num_edges=327660,
      ndata_schemes={'x': Scheme(shape=(3,), dtype=torch.float32)}
      edata_schemes={'x': Scheme(shape=(4,), dtype=torch.float32)})
2025-02-19 17:23:43,684 [INFO ]  Creating grid2mesh bipartite graph
2025-02-19 17:23:43,684 [INFO ]  Creating grid2mesh bipartite graph
2025-02-19 17:23:43,730 [INFO ]  Found max edge length = 0.020673044140092828
2025-02-19 17:23:43,730 [INFO ]  Found max edge length = 0.020673044140092828
2025-02-19 17:23:43,780 [INFO ]  mesh graph=Graph(num_nodes=39652, num_edges=316978,
      ndata_schemes={'x': Scheme(shape=(3,), dt

In [17]:
graph2mesh

Graph(num_nodes={'grid': 33, 'mesh': 39652},
      num_edges={('grid', 'g2m', 'mesh'): 47},
      metagraph=[('grid', 'mesh', 'g2m')])

In [18]:
graph_mesh

Graph(num_nodes=39652, num_edges=316978,
      ndata_schemes={'x': Scheme(shape=(3,), dtype=torch.float32), '_ID': Scheme(shape=(), dtype=torch.int32)}
      edata_schemes={'x': Scheme(shape=(4,), dtype=torch.float32), '_ID': Scheme(shape=(), dtype=torch.int32)})

In [19]:
#plot_dgl_hetero_graph_3d(graph2mesh, ('grid', 'mesh'), 'g2m')